In [2]:
#!git init   Only executed one time

In [6]:
!dvc init -f

Initialized DVC repository.

You can now commit the changes to git.

+---------------------------------------------------------------------+
|                                                                     |
|        DVC has enabled anonymous aggregate usage analytics.         |
|     Read the analytics documentation (and how to opt-out) here:     |
|             <https://dvc.org/doc/user-guide/analytics>              |
|                                                                     |
+---------------------------------------------------------------------+

What's next?
------------
- Check out the documentation: <https://dvc.org/doc>
- Get help and share ideas: <https://dvc.org/chat>
- Star us on GitHub: <https://github.com/iterative/dvc>


In [9]:
!dvc add ./wine_quality.csv

⠋ Checking graph                                                   ⠋ Checking graph
Adding...                                                                       
!
  0% Checking cache in '/workspace/mlops101-wine/.dvc/cache'| |0/? [00:00<?,    
                                                                                
!
  0%|          |Transferring                          0/1 [00:00<?,     ?file/s]
                                                                                
!
  0%|          |fbffe83c746612cc247b182e9f7278     0.00/? [00:00<?,        ?B/s]
  0%|          |fbffe83c746612cc247b182e9f7278  0.00/258k [00:00<?,        ?B/s]
100% Adding...|████████████████████████████████████████|1/1 [00:00,  4.99file/s]

To track the changes with git, run:

    git add wine_quality.csv.dvc

To enable auto staging, run:

	dvc config core.autostage true


In [10]:
!git add wine_quality.csv.dvc

In [6]:
!git add .

In [11]:
!git commit -m "dvc init"

[master 65c1d5b] dvc init
 1 file changed, 2 insertions(+), 2 deletions(-)


In [12]:
!python src/get_data.py

In [13]:
!git add . && git commit -m "added get_data.py"

[master ffa9796] added get_data.py
 7 files changed, 5 insertions(+), 514 deletions(-)
 delete mode 100644 .dvc/plots/confusion.json
 delete mode 100644 .dvc/plots/confusion_normalized.json
 delete mode 100644 .dvc/plots/linear.json
 delete mode 100644 .dvc/plots/scatter.json
 delete mode 100644 .dvc/plots/simple.json
 delete mode 100644 .dvc/plots/smooth.json


In [14]:
!python src/load_data.py 

In [15]:
!git add . && git commit -m "added load_data.py"

[master 191eaea] added load_data.py
 1 file changed, 0 insertions(+), 0 deletions(-)
 create mode 100644 src/__pycache__/get_data.cpython-310.pyc


In [17]:
!dvc run --force -n load_data \
    -d src/load_data.py \
    -d src/get_data.py \
    -d wine_quality.csv \
    -o data/raw/winequality.csv \
    python src/load_data.py --config_path=params.yaml 

Running stage 'load_data':                                            core>
> python src/load_data.py --config_path=params.yaml
Modifying stage 'load_data' in 'dvc.yaml'                                       
Updating lock file 'dvc.lock'

To track the changes with git, run:

    git add dvc.lock dvc.yaml

To enable auto staging, run:

	dvc config core.autostage true


In [18]:
!git add dvc.yaml dvc.lock data/raw/.gitignore

In [19]:
!ls 
!cat dvc.yaml
!cat dvc.lock

README.md	 dvc.lock   models	 requirements.txt  wine_quality.csv
artifacts	 dvc.yaml   notebooks	 setup.ipynb	   wine_quality.csv.dvc
cookiecutter.py  mlflow.db  params.yaml  src
data		 mlruns     reports	 train.py
stages:
  load_data:
    cmd: python src/load_data.py --config_path=params.yaml
    deps:
    - src/get_data.py
    - src/load_data.py
    - wine_quality.csv
    outs:
    - data/raw/winequality.csv
  split_data:
    cmd: python src/split_data.py --config_path=params.yaml
    deps:
    - data/raw/winequality.csv
    - src/split_data.py
    outs:
    - data/processed/test_winequality.csv
    - data/processed/train_winequality.csv
  train_and_evaluate:
    cmd: python src/train_and_evaluate.py --config_path=params.yaml
    deps:
    - data/processed/test_winequality.csv
    - data/processed/train_winequality.csv
    - src/train_and_evaluate.py
    params:
    - estimators.ElasticNet.params.alpha
    - estimators.ElasticNet.params.l1_ratio
    metrics:
    - reports/scores.json

In [3]:
import pandas as pd

In [6]:
raw_data = pd.read_csv('data/raw/winequality.csv')
raw_data.describe().T

,count,mean,std,min,25%,50%,75%,max
fixed_acidity,1599.0,8.319637,1.741096,4.60000,7.1000,7.90000,9.200000,15.90000
volatile_acidity,1599.0,0.527821,0.179060,0.12000,0.3900,0.52000,0.640000,1.58000
citric_acid,1599.0,0.270976,0.194801,0.00000,0.0900,0.26000,0.420000,1.00000
residual_sugar,1599.0,2.538806,1.409928,0.90000,1.9000,2.20000,2.600000,15.50000
chlorides,1599.0,0.087467,0.047065,0.01200,0.0700,0.07900,0.090000,0.61100
free_sulfur_dioxide,1599.0,15.874922,10.460157,1.00000,7.0000,14.00000,21.000000,72.00000
total_sulfur_dioxide,1599.0,46.467792,32.895324,6.00000,22.0000,38.00000,62.000000,289.00000
density,1599.0,0.996747,0.001887,0.99007,0.9956,0.99675,0.997835,1.00369
pH,1599.0,3.311113,0.154386,2.74000,3.2100,3.31000,3.400000,4.01000
sulphates,1599.0,0.658149,0.169507,0.33000,0.5500,0.62000,0.730000,2.00000


In [19]:
!dvc run -n split_data \
    -d src/split_data.py \
    -d data/raw/winequality.csv \
    -o data/processed/train_winequality.csv \
    -o data/processed/test_winequality.csv \
    python src/split_data.py --config_path=params.yaml 

Running stage 'split_data':
> python src/split_data.py --config_path=params.yaml
hi
      fixed_acidity  volatile_acidity  citric_acid  ...  sulphates  alcohol  quality
803             7.7              0.56         0.08  ...       0.66      9.6        6
124             7.8              0.50         0.17  ...       0.48      9.5        5
350            10.7              0.67         0.22  ...       0.98      9.9        6
682             8.5              0.46         0.31  ...       0.54      9.8        5
1326            6.7              0.46         0.24  ...       0.60     10.6        6

[5 rows x 12 columns]
     fixed_acidity  volatile_acidity  citric_acid  ...  sulphates  alcohol  quality
493            8.7             0.690         0.31  ...       0.74     11.6        6
354            6.1             0.210         0.40  ...       0.59     11.9        6
342           10.9             0.390         0.47  ...       0.75      9.8        6
834            8.8             0.685         0.

In [20]:
!git add dvc.yaml dvc.lock data/processed/.gitignore

In [22]:
!dvc repro

'wine_quality.csv.dvc' didn't change, skipping
Stage 'load_data' didn't change, skipping
Running stage 'split_data':
> python src/split_data.py --config_path=params.yaml
Updating lock file 'dvc.lock'

To track the changes with git, run:

	git add dvc.lock
Use `dvc push` to send your updates to remote storage.


In [13]:
!dvc run -n train_and_evaluate \
    -d src/train_and_evaluate.py \
    -d data/processed/train_winequality.csv \
    -d data/processed/test_winequality.csv \
    -p estimators.ElasticNet.params.alpha \
    -p estimators.ElasticNet.params.l1_ratio \
    -o models/model.joblib \
    python src/train_and_evaluate.py --config_path=params.yaml 

Running stage 'train_and_evaluate':
> python src/train_and_evaluate.py --config_path=params.yaml
Elasticnet model (alpha=0.900000, l1_ratio=0.400000):
  RMSE: 0.8031192316982254
  MAE: 0.6551462834843995
  R2: 0.013014969842177981
Adding stage 'train_and_evaluate' in 'dvc.yaml'
Updating lock file 'dvc.lock'

To track the changes with git, run:

	git add models/.gitignore dvc.lock dvc.yaml


In [ ]:
!mlflow server --backend-store-uri sqlite:///mlflow.db --default-artifact-root ./artifacts --host 127.0.0.1 -p 1234

In [1]:
!dvc repro

'wine_quality.csv.dvc' didn't change, skipping
Stage 'load_data' didn't change, skipping
Stage 'split_data' didn't change, skipping
Running stage 'train_and_evaluate':
> python src/train_and_evaluate.py --config_path=params.yaml
Updating lock file 'dvc.lock'

To track the changes with git, run:

	git add dvc.lock
Use `dvc push` to send your updates to remote storage.


In [4]:
!dvc params diff

Path         Param                                  HEAD                                  workspace
params.yaml  base.target_col                        -                                     quality
params.yaml  base.target_state                      TARGET                                -
params.yaml  estimators.ElasticNet.params.alpha     0.9                                   0.88
params.yaml  estimators.ElasticNet.params.l1_ratio  0.4                                   0.89
params.yaml  mlflow_config.artifacts_dir            -                                     artifacts
params.yaml  mlflow_config.experiment_name          -                                     ElasticNet regression
params.yaml  mlflow_config.registered_model         -                                     ElasticNetWineModel
params.yaml  mlflow_config.remote_server_uri        -                                     http://0.0.0.0:1234
params.yaml  mlflow_config.run_nane                 -                                   

In [2]:
!git add . && git commit -m "added mlflow"

[master 027e92a] added mlflow
 71 files changed, 1053 insertions(+), 13 deletions(-)
 create mode 100644 artifacts/1/451a88acdcdc431b9b51f4bb665d8a9d/artifacts/model/MLmodel
 create mode 100644 artifacts/1/451a88acdcdc431b9b51f4bb665d8a9d/artifacts/model/conda.yaml
 create mode 100644 artifacts/1/451a88acdcdc431b9b51f4bb665d8a9d/artifacts/model/model.pkl
 create mode 100644 artifacts/1/451a88acdcdc431b9b51f4bb665d8a9d/artifacts/model/requirements.txt
 create mode 100644 artifacts/1/7f98918f43f74203a75ccd05ac79f8f6/artifacts/model/MLmodel
 create mode 100644 artifacts/1/7f98918f43f74203a75ccd05ac79f8f6/artifacts/model/conda.yaml
 create mode 100644 artifacts/1/7f98918f43f74203a75ccd05ac79f8f6/artifacts/model/model.pkl
 create mode 100644 artifacts/1/7f98918f43f74203a75ccd05ac79f8f6/artifacts/model/requirements.txt
 create mode 100644 data/processed/.gitignore
 create mode 100644 data/raw/.gitignore
 create mode 100644 dvc.lock
 create mode 100644 mlflow.db
 create mode 100644 mlruns/0/m

In [6]:
!dvc run -n tag_models \
    -d src/tag_models.py \
    python src/tag_models.py --config_path=params.yaml 

Running stage 'tag_models':
> python src/tag_models.py --config_path=params.yaml
<Experiment: artifact_location='file:///workspace/mlops101-wine/mlruns/0', experiment_id='0', lifecycle_stage='active', name='Default', tags={}>
Name: Default
Experiment_id: 0
Artifact Location: file:///workspace/mlops101-wine/mlruns/0
Tags: {}
Lifecycle_stage: active
Adding stage 'tag_models' in 'dvc.yaml'
Updating lock file 'dvc.lock'

To track the changes with git, run:

	git add dvc.lock dvc.yaml
